In [1]:
import json

In [2]:
QueryDocs_Xia = json.load(open('/home/zengwei/data/XiaLong/query_doc.json'))

QueryDocs={}
for query in QueryDocs_Xia.keys():
    
    QueryDocs[query] = []
    for doc in QueryDocs_Xia[query].keys():
        QueryDocs[query].append(doc)

In [3]:
QueryDocumentTopic = {}
for query in QueryDocs_Xia.keys():
    QueryDocumentTopic[query]={}
    
    for doc in QueryDocs_Xia[query].keys():
        if len(QueryDocs_Xia[query][doc]) >0:
            
            QueryDocumentTopic[query][doc] = QueryDocs_Xia[query][doc]

In [4]:
Truth = {}
for query in QueryDocs_Xia.keys():
    Truth[query]=[]
    Truth[query].append({})
    
    subtopic_list = []
    for doc in QueryDocs_Xia[query].keys():
        if len(QueryDocs_Xia[query][doc]) >0:
            Truth[query][0][doc]= []
            for subtopic in QueryDocs_Xia[query][doc]:
                Truth[query][0][doc].append((subtopic, 1))
                if subtopic not in subtopic_list:
                    subtopic_list.append(subtopic)
                
    Truth[query].append(len(subtopic_list))

In [6]:
Doc2Vec_doc   = json.load(open('/home/zengwei/data/XiaLong/doc_representation.dat'))
Doc2Vec_query = json.load(open('/home/zengwei/data/XiaLong/query_representation.dat'))
Doc2Vec =dict (Doc2Vec_doc, **Doc2Vec_query)

In [6]:
import tensorflow as tf
from Exp_gain import *

def init_vec(dimension):
    return tf.Variable(tf.random_normal(dimension, stddev=0.01))


class DiverseRank(object):
    def __init__(self, lenepisode, nfeature, nhidden, lr):

        self.len_episode = lenepisode
        self.lr = lr
        
        global learning_rate, query_docs, candidate_docs, score, policy, train_step, sess

        with tf.name_scope('input'):
            learning_rate = tf.placeholder(tf.float32)
            query_docs = tf.placeholder(tf.float32, [None, nfeature], name='docs')
            candidate_docs = tf.placeholder(tf.float32, [None, nfeature], name='candidate_docs')

        cell = tf.contrib.rnn.GRUCell(nhidden)

        with tf.name_scope('rnn'):
            ep_split = tf.split(query_docs, 1, 0, 'split')
            _, state = tf.contrib.rnn.static_rnn(cell, ep_split, dtype=tf.float32)

        with tf.name_scope('policy'):
            w = init_vec([nfeature, nhidden])
            score = tf.tanh(tf.matmul(candidate_docs, tf.matmul(w, tf.reshape(state[-1], [nhidden, 1]))))
            policy = tf.nn.softmax(tf.transpose(score))

        init = tf.global_variables_initializer()
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=policy, labels=[0])

        train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

        sess = tf.Session()
        sess.run(init)



    def gen_episode_softmax(self, queryvec, docvec, docs):
        ndoc = len(docvec)
        
        docd_id = docs[:]
        n_candidate = ndoc

        rank_list = []
        for ite in range(min(ndoc, self.len_episode)):
            prob = sess.run(policy, feed_dict={query_docs: np.asanyarray(queryvec), candidate_docs: np.asanyarray(docvec)})
            action = np.random.choice(n_candidate, 1, p=prob[0])[0]

            n_candidate = n_candidate - 1
            rank_list.append(docd_id[action])
            queryvec.append(docvec[action])
            del docd_id[action]
            del docvec[action]

        return rank_list, queryvec + docvec


    def gen_episode_greedy(self, queryvec, docvec, docs):

        Ndoc = len(docvec)
        Nite = min(Ndoc, self.len_episode)
        
        doc_id = docs[:]
        
        
        rank_list = []
        for ite in range(Nite):
            scores = sess.run(score, feed_dict={query_docs: np.asanyarray(queryvec), candidate_docs: np.asanyarray(docvec)})
            action = np.argmax(scores)

            rank_list.append(doc_id[action])
            queryvec.append(docvec[action])
            del doc_id[action]
            del docvec[action]

        return rank_list, queryvec + docvec


    def test_model(self, querydocs, doc2vec, truth):
        meanreward = 0
        for query in querydocs.keys():
            docs = querydocs[query][:]

            ndoc = len(docs)
            if ndoc < 2:
                print '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!', Query
                continue

            querydocsvec = []
            querydocsvec.append(doc2vec[query])
            docsvec = []
            for doc in docs:
                docsvec.append(doc2vec[doc])

            # gen episode
            ranklist, _= self.gen_episode_greedy(querydocsvec, docsvec, docs)
            
            reward = alphaDCG_get_reward(ranklist, truth[query])
            # print reward[0],
            meanreward += reward[0]

        print '\n               ', meanreward, '\n'


    def train_model(self, querydocs, doc2vec, truth):
        querylist = querydocs.keys()
        train_data = {}
        for query in querylist:
            docs = querydocs[query][:]

            querydocsvec = []
            querydocsvec.append(doc2vec[query])
            docsvec = []
            for doc in docs:
                docsvec.append(doc2vec[doc])

            ranklist, querydocvec = self.gen_episode_softmax(querydocsvec, docsvec, docs)
            # print ranklist
            # print truth[query]
            reward = alphaDCG_get_reward(ranklist, truth[query])
            # print 'reward', reward[0]
            train_data[query] = { 'querydocvec': querydocvec, 'reward' : reward }

        for query in querylist:
            querydocvec = train_data[query]['querydocvec']
            reward = train_data[query]['reward']
            self.optimize_model(querydocvec, reward)


    def optimize_model(self, querydocvec, reward):
        ndoc = len(querydocvec)
        for i in range(len(reward)):
            if reward[i]==0:
                continue
            sess.run(train_step, feed_dict={query_docs: np.asanyarray(querydocvec[0:i + 1]),candidate_docs: np.asanyarray(querydocvec[i + 1:ndoc + 1]),learning_rate: self.lr * reward[i]})

    def main(self, nite, querydocs, doc2vec, query_document_topic):
        for ite in range(nite):
            self.train_model(querydocs, doc2vec, query_document_topic)

            if ite % 1 == 0:
                self.test_model(querydocs, doc2vec, query_document_topic)

In [7]:
Nfeature=100
Nhidden = 10
Lenepisode = 50
lr=-0.0001

model = DiverseRank(Lenepisode, Nfeature, Nhidden, lr)


model.main(1000, QueryDocs, Doc2Vec, Truth)


                248.268350163 




                248.268350163 




                248.268350163 




                248.268350163 




                248.268350163 




                248.257276856 



KeyboardInterrupt: 